In [1]:
import setproctitle

setproctitle.setproctitle("alex_corr")

In [2]:
import gc
import os
import sys
import time

import catboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

tqdm.pandas()

In [3]:
cat_features = [
    "emp_title",
    "home_ownership",
    "verification_status",
    "purpose",
    "zip_code",
    "addr_state",
    "initial_list_status",
    "application_type",
    "verification_status_joint",
    "disbursement_method",
]

In [4]:
samples = {
    k: pd.read_parquet(f"/home/asciishell/s3/jupyter.asciishell.ru/data_{k}.parquet") for k in ["train", "oos", "oot"]
}

In [22]:
features = sorted(set(samples["train"].columns) - {"target", "issue_d", "last_pymnt_d", "next_pymnt_d", "int_rate", "term", "installment", "funded_amnt", "loan_amnt","dti"})

In [23]:
pools = {
    k: catboost.Pool(v[v["target"] != -1][features], v[v["target"] != -1]["int_rate"], cat_features=cat_features)
    for k, v in samples.items()
}

# Chec

In [24]:
clf = catboost.CatBoostRegressor(
    iterations=400,
    max_depth=4,
    learning_rate=0.1,
    rsm=0.7,
    od_wait=100,
    use_best_model=True,
    eval_metric="R2:hints=skip_train~false",
    logging_level="Silent",
)
clf.fit(
    pools["train"],
    eval_set=pools["oos"],
    verbose=0,
    plot=True,
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

{'R2': 0.4632472581320498, 'RMSE': 3.3468170932686387}

In [25]:
clf.get_feature_importance(prettified=True)

,Feature Id,Importances
0,funded_amnt_inv,15.808037
1,fico_range_low,14.309293
2,debt_load,13.119340
3,fico_range_high,12.016103
4,purpose,7.356514
5,bc_open_to_buy,5.854949
6,verification_status,4.764321
7,initial_list_status,2.456432
8,annual_inc,2.396423
9,log_annual_inc,2.214617
